## Open Source Modelling with BBC News Dataset

For comparison of Summary in different aspects/topics.

Understand LLM Strengths and Weaknesses:

- **Identify domain-specific strengths**: Different LLMs are trained on different datasets and may excel in different domains. Comparing summaries across topics can help you identify which LLM performs best in a specific area relevant to your needs.

- **Uncover biases and limitations**: LLMs can inherit biases from their training data. Comparing summaries can help you identify potential biases and limitations in different models, allowing you to choose the one with the least bias for your task.

- **Evaluate factual accuracy**: Some LLMs prioritize fluency over factual accuracy, while others excel at fact-checking. Comparing summaries can help you assess the factual accuracy of each LLM and choose the one that best suits your need for reliable information.

Steps:

1. **Install & Import Necessary Libraries**

2. **Helper Function**

3. **Load Dataset (BBC News)** : Pick 5 rows of data per aspects

4. **Top 5 Open Source Model Summarizer**

#### Install & Import Necessary Libraries

In [1]:
import time
import sys
sys.path.append('../')
from helper.SummarizationMetrics import SummarizationMetrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import ast
import seaborn as sns


import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer, util
from scipy.signal import argrelextrema
from sklearn.cluster import KMeans


import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

c:\Users\Zhang Xiang\Desktop\Year 3\Sem 2\FYPJ\Project\RD_TextSummarizerForWebinar\Development\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

#### Helper Function

In [2]:
def summ_pipeline(model, tokenizer, chain_type, max_length, prompt=False):
  pipeline = transformers.pipeline(
      "summarization",
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=max_length,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )
  llm = HuggingFacePipeline(pipeline = pipeline)

  if chain_type == "map_reduce":
    if prompt:
      prompt_template = """Summarize this: ```{text}```"""
      prompt_message = PromptTemplate(template=prompt_template, input_variables=["text"])
      
      summary_chain = load_summarize_chain(llm=llm, chain_type=chain_type, token_max=max_length, prompt=prompt_message)
    else:
      summary_chain = load_summarize_chain(llm=llm, chain_type=chain_type, token_max=max_length)
  else:
    # can't get it to work with refine and stuff, think they updated the library but no documentation
    # on how to set token_max
    summary_chain = load_summarize_chain(llm=llm, chain_type=chain_type)
  return summary_chain

### BBC DATASET
#### Load Dataset

In [3]:
bbc_train_df = pd.read_excel("../Data/newsbbc_train.xlsx")

bbc_train_df.head()

,File_path,Articles,Summaries,transcript,summary
0,business,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...
1,politics,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...
2,entertainment,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...
3,politics,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...
4,politics,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ..."


Get 5 records of data for each topics.

In [4]:
# Randomly get 5 records from each topics.
bbc_train_df = bbc_train_df.groupby('File_path').head(5)
bbc_train_df['File_path'].value_counts()

File_path
business         5
politics         5
entertainment    5
sport            5
tech             5
Name: count, dtype: int64

In [5]:
bbc_train_df

,File_path,Articles,Summaries,transcript,summary
0,business,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...
1,politics,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...
2,entertainment,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...
3,politics,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...
4,politics,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ..."
5,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
6,sport,Chelsea denied by James heroics..A brave defen...,Chelsea were now looking more like Premiership...,Chelsea denied by James heroics..A brave defen...,Chelsea were now looking more like Premiership...
7,politics,Guantanamo man 'suing government'..A British t...,He said he was sent there after being interrog...,Guantanamo man 'suing government'..A British t...,He said he was sent there after being interrog...
13,business,Could Yukos be a blessing in disguise?..Other ...,But it argues that more rigorous tax policing ...,Could Yukos be a blessing in disguise?..Other ...,But it argues that more rigorous tax policing ...
14,business,Asian quake hits European shares..Shares in Eu...,The unfolding scale of the disaster in south A...,Asian quake hits European shares..Shares in Eu...,The unfolding scale of the disaster in south A...


When running too much models at once it could take very long, also Kernel will stop.

Recommend: Run model separately.

In [6]:
# df_scores = pd.read_excel("../Process/result/open_source_model_topics_comparison.xlsx")

# Defined the top 5 Models
data = {
    'Models': [
        # 'pszemraj/led-base-book-summary',
        # 'pszemraj/led-large-book-summary',
        # 'HHousen/distil-led-large-cnn-16384',
        'philschmid/bart-large-cnn-samsum',
        'pszemraj/long-t5-tglobal-base-16384-book-summary'
    ]
}

top_5_models = pd.DataFrame(data)
top_5_models

,Models
0,philschmid/bart-large-cnn-samsum
1,pszemraj/long-t5-tglobal-base-16384-book-summary


In [7]:
# Create DataFrame to store summary result and performance.
df_scores = pd.DataFrame(columns=['model', 'method', 'max_tokens', "topic" ,'transcript','original summary', 'summary', 'rouge','bert_score', 'bleu', 'time_taken', 'grammar', 'readability'])
df_scores

,model,method,max_tokens,topic,transcript,original summary,summary,rouge,bert_score,bleu,time_taken,grammar,readability


In [8]:
# Read file, if want to continue add on summary result.
df_scores = pd.read_excel("../Process/result/open_source_model_topics_comparison.xlsx")
print(df_scores.shape)
df_scores.head()

(75, 13)


,model,method,max_tokens,topic,transcript,original summary,summary,rouge,bert_score,bleu,time_taken,grammar,readability
0,pszemraj/led-base-book-summary,MapReduce,16384,business,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,The following is a summary of the announcement...,"[{'rouge-1': {'r': 0.38095238095238093, 'p': 0...","(tensor([0.8857]), tensor([0.8723]), tensor([0...",0.218163,19.899436,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 13.012869198312234, grade_level: '13'"
1,pszemraj/led-base-book-summary,MapReduce,16384,politics,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,"In this brief summary, we summarize the speech...","[{'rouge-1': {'r': 0.2874251497005988, 'p': 0....","(tensor([0.8664]), tensor([0.8317]), tensor([0...",0.095036,21.163034,"[Match({'ruleId': 'EN_COMPOUNDS', 'message': '...","score: 9.665454545454548, grade_level: '10'"
2,pszemraj/led-base-book-summary,MapReduce,16384,entertainment,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,The narrator of this piece originally appeared...,"[{'rouge-1': {'r': 0.4148148148148148, 'p': 0....","(tensor([0.8524]), tensor([0.8609]), tensor([0...",0.175990,28.523080,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 11.396671501087742, grade_level: '11'"
3,pszemraj/led-base-book-summary,MapReduce,16384,politics,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,"The Home Secretary, Charles Clarke, has outlin...","[{'rouge-1': {'r': 0.2710843373493976, 'p': 0....","(tensor([0.8730]), tensor([0.8468]), tensor([0...",0.056949,24.956480,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 11.617224157955867, grade_level: '12'"
4,pszemraj/led-base-book-summary,MapReduce,16384,politics,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",The English Parliament passes the Prevention o...,"[{'rouge-1': {'r': 0.3959731543624161, 'p': 0....","(tensor([0.8890]), tensor([0.8720]), tensor([0...",0.167719,21.425882,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 9.484285714285715, grade_level: '9'"


#### Top 5 Open Source Model Summarizer

In [9]:
for model_name in top_5_models['Models']:
    print(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    print(tokenizer.model_max_length)
    for index, row in bbc_train_df.iterrows():
        method = "MapReduce"
        
        # get the summary
        start_time = time.time()


        max_tokens = tokenizer.model_max_length
        summary_chain = summ_pipeline(model, tokenizer, "map_reduce", max_tokens)

        # Used for efficient tokenization and processing of long texts when working with language models
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=max_tokens-100, chunk_overlap=100)
        docs = text_splitter.create_documents([row["transcript"]])
        summary = summary_chain.run(docs)


        end_time = time.time()
        elapsed_time = end_time - start_time

        metrics = SummarizationMetrics(row['summary'], summary)

        new_result = {
            'model': model_name,
            'method': method,
            'max_tokens': max_tokens,
            'topic': row["File_path"],
            'transcript': row['transcript'],
            'original summary': row['summary'],
            'summary': summary,
            'rouge': metrics.rouge_scores(),
            'bert_score': metrics.bert_score(),
            'bleu': metrics.bleu_score(),
            'time_taken': elapsed_time,
            'grammar': metrics.grammar_check(),
            'readability': metrics.readability_index()
        }


        new_row = pd.DataFrame([new_result])

        df_scores = pd.concat([df_scores, new_row], ignore_index=True)


philschmid/bart-large-cnn-samsum
1024


Your max_length is set to 1024, but your input_length is only 785. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=392)
Your max_length is set to 1024, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


computing greedy matching.


...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 166.78it/s]
c:\Users\Zhang Xiang\Desktop\Year 3\Sem 2\FYPJ\Project\RD_TextSummarizerForWebinar\Development\.venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


done in 2.61 seconds, 0.38 sentences/sec


In [10]:
df_scores

,model,method,max_tokens,topic,transcript,original summary,summary,rouge,bert_score,bleu,time_taken,grammar,readability
0,pszemraj/led-base-book-summary,MapReduce,16384,business,Cuba winds back economic clock..Fidel Castro's...,Fidel Castro's decision to ban all cash transa...,The following is a summary of the announcement...,"[{'rouge-1': {'r': 0.38095238095238093, 'p': 0...","(tensor([0.8857]), tensor([0.8723]), tensor([0...",2.181628e-01,19.899436,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 13.012869198312234, grade_level: '13'"
1,pszemraj/led-base-book-summary,MapReduce,16384,politics,Blair looks to election campaign..Tony Blair's...,There was little in terms of concrete proposal...,"In this brief summary, we summarize the speech...","[{'rouge-1': {'r': 0.2874251497005988, 'p': 0....","(tensor([0.8664]), tensor([0.8317]), tensor([0...",9.503589e-02,21.163034,"[Match({'ruleId': 'EN_COMPOUNDS', 'message': '...","score: 9.665454545454548, grade_level: '10'"
2,pszemraj/led-base-book-summary,MapReduce,16384,entertainment,New York rockers top talent poll..New York ele...,New York electro-rock group The Bravery have c...,The narrator of this piece originally appeared...,"[{'rouge-1': {'r': 0.4148148148148148, 'p': 0....","(tensor([0.8524]), tensor([0.8609]), tensor([0...",1.759903e-01,28.523080,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 11.396671501087742, grade_level: '11'"
3,pszemraj/led-base-book-summary,MapReduce,16384,politics,Terror suspects face house arrest..UK citizens...,British citizens are being included in the cha...,"The Home Secretary, Charles Clarke, has outlin...","[{'rouge-1': {'r': 0.2710843373493976, 'p': 0....","(tensor([0.8730]), tensor([0.8468]), tensor([0...",5.694877e-02,24.956480,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 11.617224157955867, grade_level: '12'"
4,pszemraj/led-base-book-summary,MapReduce,16384,politics,'No more concessions' on terror..Charles Clark...,"On Monday, MPs voted 272-219 in favour of the ...",The English Parliament passes the Prevention o...,"[{'rouge-1': {'r': 0.3959731543624161, 'p': 0....","(tensor([0.8890]), tensor([0.8720]), tensor([0...",1.677187e-01,21.425882,"[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'me...","score: 9.484285714285715, grade_level: '9'"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,pszemraj/long-t5-tglobal-base-16384-book-summary,MapReduce,1000000000000000019884624838656,sport,Paris promise raises Welsh hopes..Has there be...,But since they threw off the shackles against ...,This brief paper gives an update on the progre...,"[{'rouge-1': {'r': 0.08695652173913043, 'p': 0...","([tensor(0.8620)], [tensor(0.8025)], [tensor(0...",4.201394e-157,9.210987,[],100 words required.
121,pszemraj/long-t5-tglobal-base-16384-book-summary,MapReduce,1000000000000000019884624838656,entertainment,Rapper Kanye West's shrewd soul..US hip-hop st...,Leaving his Chicago art school after only one ...,The Narrator informs the audience that Kany We...,"[{'rouge-1': {'r': 0.09392265193370165, 'p': 0...","([tensor(0.8522)], [tensor(0.7918)], [tensor(0...",1.260210e-156,15.023796,"[Offset 39, length 4, Rule ID: MORFOLOGIK_RULE...",100 words required.
122,pszemraj/long-t5-tglobal-base-16384-book-summary,MapReduce,1000000000000000019884624838656,entertainment,Redford's vision of Sundance..Despite sporting...,Redford wanted Sundance to be a platform for i...,Robert Redford is the founder and president of...,"[{'rouge-1': {'r': 0.07096774193548387, 'p': 0...","([tensor(0.8724)], [tensor(0.8100)], [tensor(0...",8.507861e-159,8.595882,"[Offset 51, length 8, Rule ID: MORFOLOGIK_RULE...",100 words required.
123,pszemraj/long-t5-tglobal-base-16384-book-summary,MapReduce,1000000000000000019884624838656,business,Water firm Suez in Argentina row..A conflict b...,The government has rejected the 60% rise and w...,This paper describes a dispute between the Fre...,"[{'rouge-1': {'r': 0.07643312101910828, 'p': 0...","([tensor(0.8490)], [t

In [11]:
# df_scores.to_excel("../Process/result/open_source_model_topics_comparison.xlsx", index=False)

In [13]:
df_scores.shape

(125, 13)